<a href="https://colab.research.google.com/github/RodrigoLima82/Tensorflow2.0/blob/master/Treinamento_Distribu%C3%ADdo_com_TensorFlow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 1: Instalação das bibliotecas

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 43kB/s 
     |████████████████████████████████| 419kB 40.6MB/s 
     |████████████████████████████████| 3.0MB 46.1MB/s 


## Etapa 2: Importação das bibliotecas

In [2]:
import time
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [3]:
tf.__version__

'2.0.0-alpha0'

## Etapa 3: Pré-processamento

### Carregando a base de dados MNIST

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Normalização das imagens

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

In [6]:
X_train.shape

(60000, 28, 28)

### Mudança das dimensões

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [8]:
X_train.shape

(60000, 784)

## Etapa 4: Treinamento distribuído

### Definição do modelo normal (não distribuído)

In [0]:
model_normal = tf.keras.models.Sequential()
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model_normal.add(tf.keras.layers.Dropout(rate=0.2))
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Definição da estratégia distribuída

In [10]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0


### Definindo um modelo distribuído

In [11]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

### Comparação de velocidade entre o treinamento normal e o treinamento distribuído

In [12]:
starting_time = time.time()
model_distributed.fit(X_train, y_train, epochs=20, batch_size=128)
print("Distributed training: {}".format(time.time() - starting_time))

INFO:tensorflow:batch_all_reduce invoked for batches size = 4 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
Epoch 1/20
469/469 [==============================] - 3s 5ms/step - loss: 0.4015 - sparse_categorical_accuracy: 0.8862
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1924 - sparse_categorical_accuracy: 0.9443
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1454 - sparse_categorical_accuracy: 0.9575
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1194 - sparse_categorical_accuracy: 0.9648
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0995 - sparse_categorical_accuracy: 0.9694
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0875 - sparse_categorical_accuracy: 0.9739
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0776 - sparse_categorical_accuracy: 0.9764
Ep

In [13]:
starting_time = time.time()
model_normal.fit(X_train, y_train, epochs=20, batch_size=128)
print("Normal training: {}".format(time.time() - starting_time))

Epoch 1/20
60000/60000 [==============================] - 2s 30us/sample - loss: 0.3937 - sparse_categorical_accuracy: 0.8873
Epoch 2/20
60000/60000 [==============================] - 2s 28us/sample - loss: 0.1894 - sparse_categorical_accuracy: 0.9456
Epoch 3/20
60000/60000 [==============================] - 2s 28us/sample - loss: 0.1419 - sparse_categorical_accuracy: 0.9587
Epoch 4/20
60000/60000 [==============================] - 2s 27us/sample - loss: 0.1162 - sparse_categorical_accuracy: 0.9659
Epoch 5/20
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0997 - sparse_categorical_accuracy: 0.9706
Epoch 6/20
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0859 - sparse_categorical_accuracy: 0.9740
Epoch 7/20
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0764 - sparse_categorical_accuracy: 0.9767
Epoch 8/20
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0679 - sparse_categorical_accuracy: